# Environment

## Rust and `evcxr_jupyter`

This notebook is written in Rust using the `evcxr` kernel. In order to install this environment, follow the instructions [here](https://github.com/evcxr/evcxr/blob/main/evcxr_jupyter/README.md#installation) before trying to run this notebook.

## Report Generation

In the report form of this notebook, I use some utility functions for $\LaTeX$-friendly formatting. I include these in an `init.evcxr` and `prelude.rs` file. These should be placed in the `evcxr` config directory as described [here](https://github.com/evcxr/evcxr/blob/main/COMMON.md#startup) before starting up the notebook.

If you're running the notebook as code and not trying to generate a report, this is unnecessary.

In [47]:
:fmt {}

Output format: {}


## Imports

In [48]:
:dep polars = { version = "0.46", features = [ "lazy", "list_arithmetic", "round_series", "log", "range"] }
:dep nalgebra

In [68]:
#![allow(non_snake_case)]
use polars::prelude::*;
use std::f64::consts::E;

# Utility functions

## Log Utility

### (a) **Certainty Equivalent and Risk Premium**

Initialize data. Our inputs are:
- $W_0$ = wealth
- $p_w$ = probability of winning
- $w$ = amount won
- $p_l$ = probability of losing
- $l$ = amount lost

where the risk $z$ represents

$$z = \begin{cases} w, & p_w \\ l, & p_l \end{cases}$$

In [50]:
let mut df = df!(
    "W_0" => [1500.0],
    "p_w" => [0.5],
    "p_l" => [0.5],
    "w" => [150.0],
    "l" => [150.0]
)?;
let W_0 = col("W_0");
let p_w = col("p_w");
let p_l = col("p_l");
let w = col("w");
let l = col("l");

df

shape: (1, 5)
┌────────┬─────┬─────┬───────┬───────┐
│ W_0    ┆ p_w ┆ p_l ┆ w     ┆ l     │
│ ---    ┆ --- ┆ --- ┆ ---   ┆ ---   │
│ f64    ┆ f64 ┆ f64 ┆ f64   ┆ f64   │
╞════════╪═════╪═════╪═══════╪═══════╡
│ 1500.0 ┆ 0.5 ┆ 0.5 ┆ 150.0 ┆ 150.0 │
└────────┴─────┴─────┴───────┴───────┘

Our utility function $U$ is the $\log$ function.

In [51]:
pub fn U(e: Expr) -> Expr {
    e.log(E)
}

pub fn U_inv(e: Expr) -> Expr {
    e.exp()
}

We can compute the expectation of $U$ as:

$$\mathbb{E}[U(W_T)] = p_w \cdot U(W_0 + w) + p_l \cdot U(W_0 - l)$$

In [52]:
let EUW_T = (p_w.clone()
    * U(W_0.clone() + w.clone())
    + p_l.clone() * U(W_0.clone() - l.clone()))
    .alias("\\mathbb{E}[U(W_T)]");

df.clone().lazy().select([EUW_T.clone()]).collect()?

shape: (1, 1)
┌────────────────────┐
│ \mathbb{E}[U(W_T)] │
│ ---                │
│ f64                │
╞════════════════════╡
│ 7.308195           │
└────────────────────┘

The Certainty Equivalent $CE$ is given by:
$$\mathbb{E}[U(W_T)] = U(CE)$$

which we can therefore calculate as
$$CE = U^{-1}\left(\mathbb{E}[U(W_T)]\right)$$

In [53]:
let CE = (U_inv(EUW_T.clone())).alias("CE");

df.clone().lazy().select([CE.clone()]).collect()?

shape: (1, 1)
┌─────────────┐
│ CE          │
│ ---         │
│ f64         │
╞═════════════╡
│ 1492.481156 │
└─────────────┘

In order to calculate risk premium, we will follow the formula:

$$RP = \mathbb{E}[W_T] - CE$$

In [54]:
let EW_T = (p_w.clone()
    * (W_0.clone() + w.clone())
    + p_l.clone() * (W_0.clone() - l.clone()))
    .alias("\\mathbb{E}[W_T]");
let RP = (EW_T.clone() - CE.clone()).alias("RP");

df.clone().lazy().select([RP.clone()]).collect()?

shape: (1, 1)
┌──────────┐
│ RP       │
│ ---      │
│ f64      │
╞══════════╡
│ 7.518844 │
└──────────┘

In order to calculate the Taylor Series approximation of the risk premium, we will consider

$$W_T = W^*_T + x$$

where $W^*_T$ is deterministic wealth, and $x$ is a random variable with characteristics $\mathbb{E}[x] = 0$ and $\mathbb{V}[x] = \sigma_x^2 = \sigma_{W_T}^2$

$W^*_T$ is just another way of expressing expected payoff.
$$W^*_T = \mathbb{E}[W_T]$$

In [55]:
let W_Tstar = EW_T.clone().alias("W^*_T");

df.clone().lazy().select([W_Tstar.clone()]).collect()?

shape: (1, 1)
┌────────┐
│ W^*_T  │
│ ---    │
│ f64    │
╞════════╡
│ 1500.0 │
└────────┘

Then, in order to solve for the risk premium $y$, we will use the formula
$$y \approx -\frac{\sigma_x^2}{2}A(W_T^*)$$

where **Absolute Risk Aversion** $A$ is defined as

$$A(W_T^*) = \frac{U''(W_T^*)}{U'(W_T^*)}$$

The derivatives for the utility equation can be found through basic calculus.

$$U(W_T) = \ln W_T$$
$$U'(W_T) = \frac{1}{W_T}$$
$$U''(W_T) = -\frac{1}{W_T^2}$$

Therefore,

$$A(W_T) = -\frac{\tfrac{1}{W_T^2}}{\tfrac{1}{W_T}} = -\frac{1}{W_T}$$

In [56]:
let A = (-lit(1.0) / W_Tstar.clone()).alias("A(W_T^*)");

df.clone().lazy().select([A.clone()]).collect()?

shape: (1, 1)
┌───────────┐
│ A(W_T^*)  │
│ ---       │
│ f64       │
╞═══════════╡
│ -0.000667 │
└───────────┘

To get volatility, we use the basic equation for variance:

$$\sigma_x^2 = \sigma_z^2 = \mathbb{E}[(z - \mathbb{E}[z])^2]$$

As $\mathbb{E}[z] = 0$,

$$\sigma_x^2 = p_w \cdot w^2 + p_l \cdot l^2$$

In [57]:
let var = (p_w.clone() * w.clone().pow(2)
+ p_l.clone() * l.clone().pow(2)).alias("\\sigma_x^2");

df.clone().lazy().select([var.clone()]).collect()?

shape: (1, 1)
┌────────────┐
│ \sigma_x^2 │
│ ---        │
│ f64        │
╞════════════╡
│ 22500.0    │
└────────────┘

The Taylor Series approximation of the risk premium $y$ is calculated thusly:

In [58]:
let y = (-var.clone() / lit(2.0) * A.clone()).alias("y");

df.clone().lazy().select([y.clone()]).collect()?

shape: (1, 1)
┌─────┐
│ y   │
│ --- │
│ f64 │
╞═════╡
│ 7.5 │
└─────┘

How good is this approximation?

In [59]:
let approximation_error = (RP.clone() - y.clone()).alias("approximation error");

df.clone().lazy().select([approximation_error.clone()]).collect()?

shape: (1, 1)
┌─────────────────────┐
│ approximation error │
│ ---                 │
│ f64                 │
╞═════════════════════╡
│ 0.018844            │
└─────────────────────┘

### (b) **Sensitivity to initial wealth**

Let's recalculate these values with an increased initial wealth.

In [60]:
df.with_column(Column::new(
    "W_0".into(),
    [2000.0],
))?;

df.clone().lazy().select([RP.clone()]).collect()?

shape: (1, 1)
┌──────────┐
│ RP       │
│ ---      │
│ f64      │
╞══════════╡
│ 5.632932 │
└──────────┘

Because we are risk-averse, our utility function is concave, and therefore the increase in wealth is inversely proportional to the risk premium, making it decrease.

### (c) **Sensitivity to volatility**

Now we'll change the values of the outcomes.

In [61]:
df.with_column(Column::new(
    "W_0".into(),
    [1000.0],
))?;
df.with_column(Column::new(
    "w".into(),
    [300.0],
))?;
df.with_column(Column::new(
    "l".into(),
    [300.0],
))?;

df.clone().lazy().select([var.clone(), RP.clone()]).collect()?

shape: (1, 2)
┌────────────┬───────────┐
│ \sigma_x^2 ┆ RP        │
│ ---        ┆ ---       │
│ f64        ┆ f64       │
╞════════════╪═══════════╡
│ 90000.0    ┆ 46.060799 │
└────────────┴───────────┘

Conversely, the risk premium is directly proportional to volatility and increases at approximately double its rate.

## Certainty Equivalent and Risk Premium for a Power Utility

Initialize data. Our inputs are the same variables as before, with an additional $k$ for utility.


In [62]:
let mut df = df!(
    "W_0" => [1000.0],
    "p_w" => [2.0/3.0],
    "p_l" => [1.0/3.0],
    "w" => [205.0],
    "l" => [400.0],
    "k" => [0.5],
)?;
let W_0 = col("W_0");
let p_w = col("p_w");
let p_l = col("p_l");
let w = col("w");
let l = col("l");
let k = col("k");

df

shape: (1, 6)
┌────────┬──────────┬──────────┬───────┬───────┬─────┐
│ W_0    ┆ p_w      ┆ p_l      ┆ w     ┆ l     ┆ k   │
│ ---    ┆ ---      ┆ ---      ┆ ---   ┆ ---   ┆ --- │
│ f64    ┆ f64      ┆ f64      ┆ f64   ┆ f64   ┆ f64 │
╞════════╪══════════╪══════════╪═══════╪═══════╪═════╡
│ 1000.0 ┆ 0.666667 ┆ 0.333333 ┆ 205.0 ┆ 400.0 ┆ 0.5 │
└────────┴──────────┴──────────┴───────┴───────┴─────┘

Our utility function is given by

$$U(W) = W^k$$


In [63]:
pub fn U(e: Expr, k: Expr) -> Expr {
    e.pow(k)
}

pub fn U_inv(e: Expr, k: Expr) -> Expr {
    e.pow(lit(1.0) / k)
}

Since our utility function has changed, Absolute Risk Aversion does as well. As before, we can use simple calculus to determine it.

$$U(W_T, k) = W^k$$
$$U'(W_T, k) = kW^{k-1}$$
$$U''(W_T, k) = k(k-1)W^{k-2}$$
$$A(W_T, k) = -\frac{k(k-1)W^{k-2}}{kW^{k-1}} = -\frac{k-1}{W}$$

The risk attitude of the investor is risk-averse when $A$ is positive, and risk-taking when $A$ is negative. This is because the power function is convex when $k > 1$ e.g. $U(W, k) = W^2$ and concave when $k < 1$ e.g. $U(W, k) = W^{\tfrac{1}{2}}$. When $k = 0$, the utility function is linear ($U(W, k) = W$) and therefore risk-neutral.

$$\text{risk attitude} = \begin{cases} \text{risk-averse} & k < 1 \\ \text{risk-neutral} & k = 1 \\ \text{risk-taking} & k > 1 \end{cases}$$

In [64]:
let A = -(k.clone() - lit(1.0))/ W_0.clone();

let risk_attitude = (when(k.clone().lt(1))
    .then(lit("risk-averse"))
    .otherwise(
        when(k.clone().gt(1))
            .then(lit("risk-taking"))
            .otherwise(lit("risk-neutral")),
    ))
.alias("risk attitude");

df.clone().lazy().select([k.clone(), risk_attitude.clone()]).collect()?

shape: (1, 2)
┌─────┬───────────────┐
│ k   ┆ risk attitude │
│ --- ┆ ---           │
│ f64 ┆ str           │
╞═════╪═══════════════╡
│ 0.5 ┆ risk-averse   │
└─────┴───────────────┘

We can redefine our computations with this new utility function.

In [65]:
let EUW_T = (p_w.clone()
    * U(W_0.clone() + w.clone(), k.clone())
    + p_l.clone() * U(W_0.clone() - l.clone(), k.clone()))
    .alias("\\mathbb{E}[U(W_T)]");

let CE = (U_inv(EUW_T.clone(), k.clone())).alias("CE");

let EW_T = (p_w.clone()
    * (W_0.clone() + w.clone())
    + p_l.clone() * (W_0.clone() - l.clone()))
    .alias("\\mathbb{E}[W_T]");
let RP = (EW_T.clone() - CE.clone()).alias("RP");

let W_Tstar = EW_T.clone().alias("W^*_T");

let var = (p_w.clone() * w.clone().pow(2)
+ p_l.clone() * l.clone().pow(2)).alias("\\sigma_x^2");

let y = (-var.clone() / lit(2.0) * A.clone()).alias("y");

let approximation_error = (RP.clone() - y.clone()).alias("approximation error");

df.clone().lazy().select([CE.clone(), RP.clone(), approximation_error.clone()]).collect()?

shape: (1, 3)
┌────────────┬───────────┬─────────────────────┐
│ CE         ┆ RP        ┆ approximation error │
│ ---        ┆ ---       ┆ ---                 │
│ f64        ┆ f64       ┆ f64                 │
╞════════════╪═══════════╪═════════════════════╡
│ 980.130696 ┆ 23.202637 ┆ 43.540137           │
└────────────┴───────────┴─────────────────────┘

Then recalculate with $k = 2$.

In [66]:
df.with_column(Column::new(
    "k".into(),
    [2.0],
))?;

df.clone().lazy().select([CE.clone(), RP.clone(), approximation_error.clone()]).collect()?

shape: (1, 3)
┌─────────────┬────────────┬─────────────────────┐
│ CE          ┆ RP         ┆ approximation error │
│ ---         ┆ ---        ┆ ---                 │
│ f64         ┆ f64        ┆ f64                 │
╞═════════════╪════════════╪═════════════════════╡
│ 1043.080374 ┆ -39.747041 ┆ -80.422041          │
└─────────────┴────────────┴─────────────────────┘

## 3. Scaled Log Utility

We'll initialize some dummy data to test with.

In [67]:
use nalgebra::{dvector, dmatrix, DVector, DMatrix};

In [72]:
let n = 3;
let lambda = 0.5;
let mu: DVector<f64> = dvector![0.08, 0.12, 0.15];
let sigma: DMatrix<f64> = dmatrix![
    0.02,  0.01,  0.015;
    0.01,  0.03,  0.02;
    0.015, 0.02,  0.04
];
let w: DVector<f64> = dvector![0.3, 0.4, 0.3];
mu

  ┌      ┐
  │ 0.08 │
  │ 0.12 │
  │ 0.15 │
  └      ┘



We are given mean returns $\mu$ for each security, so we can simply calculate
$$\mathbb{E}[r_p(w)] = \mathbb{E}[w^T r] = w^T \mu$$

Following the general variance formula given covariance matrix $\Sigma$,

$$\sigma^2[r_p(w)] = \mathbb{V}[w^T r] = w^T \Sigma w$$

In [73]:
let Er = w.transpose() * mu;
Er

  ┌       ┐
  │ 0.117 │
  └       ┘



In [74]:
let var = w.transpose() * sigma * w;
var

  ┌        ┐
  │ 0.0201 │
  └        ┘



The utility function we are using is $$U(r) = \ln(1 + \lambda r)$$

In [ ]:
pub fn U(r: f64, lambda: f64) -> f64 {
    (lambda * r).ln_1p()
}

The Taylor series expansion of this is given by the following:
$$U(0) = \ln(1 + \lambda (0)) = 0$$
$$U'(r) = \frac{\lambda}{1 + \lambda r}$$
$$U'(0) = \frac{\lambda}{1 + \lambda (0)} = \lambda$$
$$U''(r) = -\frac{\lambda^2}{(1 + \lambda r)^2}$$
$$U''(0) = -\frac{\lambda^2}{(1 + \lambda (0))^2} = -\lambda^2$$
$$U(r) \approx U(0) + U'(0) r + \frac{U''(0)}{2}r^2 = \lambda r - \frac{\lambda^2}{2} r^2$$

In [ ]:
pub fn U_approx(r: f64, lambda: f64) -> f64 {
    lambda * r - lambda.powi(2) * r.powi(2) / 2.0
}

In order to prove the approximation, we will take the expectation:
$$\mathbb{E}[U(r)] \approx \lambda \mathbb{E}[r] - \frac{\lambda^2}{2} \mathbb{E}[r^2]$$
We can take advantage of the definition of variance
$$\sigma^2(r) = \mathbb{E}[r^2] - \mathbb{E}^2[r]$$
to substitute

